In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Pet dog sound event.zip to Pet dog sound event.zip


In [ ]:
import zipfile
import os

with zipfile.ZipFile("Pet dog sound event.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")  # This creates a folder named 'dataset' with the contents


In [ ]:
import librosa
import glob

# Load audio files from extracted directory
audio_files = glob.glob("dataset/*.wav")
data = [librosa.load(file_path) for file_path in audio_files]


In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load YAMNet model from TensorFlow Hub
model = hub.load('https://tfhub.dev/google/yamnet/1')


In [ ]:
pip install sounddevice numpy librosa tensorflow tensorflow-hub keras scikit-learn joblib


In [ ]:
!apt-get update
!apt-get install -y portaudio19-dev
!pip install --upgrade sounddevice

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,275 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [ ]:
import sounddevice as sd

import sounddevice as sd

# Set default input device (try 0 or any valid index from `query_devices()`)
sd.default.device = (0, None)  # (input_device_index, output_device_index)
!sudo apt-get install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving dog-bark-179915.wav to dog-bark-179915.wav


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email_notification(recipient_email, subject, message_body):
    # Email credentials (Use your own credentials)
    sender_email = "1032221997@mitwpu.edu.in"
    sender_password = "akiu wdzm yyea sarn"  # Or App Password if 2FA is enabled

    # Setup the email details
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the message body
    msg.attach(MIMEText(message_body, 'plain'))

    try:
        # Establish a secure connection with Gmail's SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Enable encryption for security
        server.login(sender_email, sender_password)  # Login to the email server

        # Send the email
        server.sendmail(sender_email, recipient_email, msg.as_string())
        print(f"Email sent to {recipient_email}")

    except Exception as e:
        print(f"Failed to send email: {e}")
    finally:
        server.quit()  # Close the SMTP connection

# Example usage
def notify_pet_mood(mood):
    recipient_email = "tanishauchakraborty@gmail.com"  # Replace with user's email
    subject = f"Your pet's current mood: {mood}"
    message_body = f"Hi! Your pet seems to be feeling {mood} right now."
    send_email_notification(recipient_email, subject, message_body)


In [ ]:
!pip install tensorflow tensorflow-hub keras scikit-learn joblib sounddevice librosa

import os
import zipfile
import numpy as np
import librosa
import glob
import tensorflow as tf
import tensorflow_hub as hub
from keras import layers, models
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import joblib
import sounddevice as sd
from tensorflow.keras.layers import Input, Lambda # Import necessary modules
from tensorflow.keras.models import Model


# Step 1: Extract Dataset from ZIP
def extract_zip(zip_file_path, extract_to):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Step 2: Load and Preprocess Audio Samples
def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)  # Resample to 16 kHz
    return signal

def preprocess_audio(signal):
    if len(signal) < 16000:
        signal = np.pad(signal, (0, 16000 - len(signal)), 'constant')
    return signal[:16000]  # Trim or pad to 1 second

def load_dataset(dataset_path):
    data = []
    labels = []

    for file in os.listdir(dataset_path):
        if file.endswith('.wav'):
            file_path = os.path.join(dataset_path, file)
            audio_signal = preprocess_audio(load_audio(file_path))
            data.append(audio_signal)

            # Map the file name to a higher-level mood (customize this mapping)
            if "growling" in file:
                labels.append("aggressive")  # Example mapping
            elif "whining" in file:
                labels.append("hungry")  # Example mapping
            elif "barking" in file:
                labels.append("happy")  # Example mapping
            else:
                # Add a default label for files that don't match the above conditions
                labels.append("unknown") # You can adjust the default label as needed


    return np.array(data), np.array(labels)

# Step 3: Encode Labels
def encode_labels(labels):
    le = LabelEncoder()
    integer_encoded = le.fit_transform(labels)
    one_hot_encoded = to_categorical(integer_encoded)
    return one_hot_encoded, le

def create_model(num_classes):
    # Load YAMNet model from TensorFlow Hub
    yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

    # Define input shape (batch_size, 16000)
    input_shape = (16000,)
    input_tensor = Input(shape=input_shape)

    # Use tf.map_fn to apply YAMNet on each sample in the batch
    def yamnet_embedding(audio_batch):
        return tf.map_fn(lambda audio: yamnet_model(audio)[1], audio_batch, dtype=tf.float32)

    # Wrap in a Lambda layer with appropriate output shape
    yamnet_output = Lambda(yamnet_embedding, output_shape=(None, 1024))(input_tensor)

    # Use Global Average Pooling to reduce the time dimension
    pooled_output = layers.GlobalAveragePooling1D()(yamnet_output)

    # Add dense layers for classification
    x = layers.Dense(128, activation='relu')(pooled_output)
    x = layers.Dropout(0.5)(x)
    output_tensor = layers.Dense(num_classes, activation='softmax')(x)

    # Create the final model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    # Freeze YAMNet layers (optional fine-tuning)
    for layer in model.layers[:-3]:  # Freeze all but the last 3 layers
        layer.trainable = False

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


# Step 5: Train the Model
def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=16):
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))
    return history


# Step 6: Real-Time Prediction
def record_audio(duration=1.0, sample_rate=16000, device=1):  # Replace 1 with your input device index
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1,
                   dtype='float32', device=device)
    sd.wait()  # Wait until the recording is complete
    print("Recording finished.")
    return audio.flatten()


def preprocess_real_time_audio(signal):
    return preprocess_audio(signal)

def predict_mood(model, label_encoder, audio_signal):
    audio_signal = preprocess_real_time_audio(audio_signal)
    audio_signal = np.expand_dims(audio_signal, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(audio_signal)
    predicted_class = np.argmax(predictions, axis=1)

    # Decode the predicted class back to label
    mood = label_encoder.inverse_transform(predicted_class)
    return mood[0]

import librosa

def main(zip_file_path, dataset_path):
    # Extract ZIP and load dataset
    extract_zip(zip_file_path, dataset_path)
    X, labels = load_dataset(dataset_path)
    y, le = encode_labels(labels)

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    num_classes = len(le.classes_)
    model = create_model(num_classes)
    history = train_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=16)

    # Save the trained model and label encoder
    model.save('dog_mood_model_finetuned.h5')
    joblib.dump(le, 'label_encoder.pkl')

    # Use a pre-recorded audio file for prediction
    signal, _ = librosa.load('dog-bark-179915.wav', sr=16000)  # Replace with a valid file path
    mood_prediction = predict_mood(model, le, signal)
    print(f"Predicted Mood: {mood_prediction}")
    notify_pet_mood(mood_prediction)

if __name__ == "__main__":
    zip_file_path = 'Pet dog sound event.zip'
    dataset_path = 'dataset/'
    main(zip_file_path, dataset_path)



Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 27s 292ms/step - accuracy: 0.4916 - loss: 1.2839 - val_accuracy: 0.7792 - val_loss: 0.7058
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 16s 260ms/step - accuracy: 0.6544 - loss: 0.7214 - val_accuracy: 0.7333 - val_loss: 0.6211
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 15s 254ms/step - accuracy: 0.7155 - loss: 0.6226 - val_accuracy: 0.9083 - val_loss: 0.5634
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 16s 268ms/step - accuracy: 0.7469 - loss: 0.6180 - val_accuracy: 0.9167 - val_loss: 0.5265
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 257ms/step - accuracy: 0.7730 - loss: 0.5671 - val_accuracy: 0.7625 - val_loss: 0.5292
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 16s 270ms/step - accuracy: 0.7531 - loss: 0.5442 - val_accuracy: 0.9125 - val_loss: 0.4896
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 258ms/step - accuracy: 0.7905 - loss: 0.4897 - val_accuracy: 0.9083 - val_loss: 0.4188
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 21s 270ms/step - accuracy: 0.7959 - loss: 0.4464 - val_accu

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step
Predicted Mood: happy
Email sent to tanishauchakraborty@gmail.com


In [ ]:
!pip install tensorflow tensorflow-hub keras scikit-learn joblib sounddevice librosa

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os
import zipfile
import numpy as np
import librosa
import glob
import tensorflow as tf
import tensorflow_hub as hub
from keras import layers, models
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import joblib
import sounddevice as sd
from tensorflow.keras.layers import Input, Lambda  # Import necessary modules
from tensorflow.keras.models import Model
from tensorflow.keras.utils import register_keras_serializable  # Import register_keras_serializable


def send_email_notification(recipient_email, subject, message_body):
    # Email credentials (Use your own credentials)
    sender_email = "1032221997@mitwpu.edu.in"
    sender_password = "akiu wdzm yyea sarn"  # Or App Password if 2FA is enabled

    # Setup the email details
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the message body
    msg.attach(MIMEText(message_body, 'plain'))

    try:
        # Establish a secure connection with Gmail's SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Enable encryption for security
        server.login(sender_email, sender_password)  # Login to the email server

        # Send the email
        server.sendmail(sender_email, recipient_email, msg.as_string())
        print(f"Email sent to {recipient_email}")

    except Exception as e:
        print(f"Failed to send email: {e}")
    finally:
        server.quit()  # Close the SMTP connection


# Example usage
def notify_pet_mood(mood):
    recipient_email = "tanishauchakraborty@gmail.com"  # Replace with user's email
    subject = f"Your pet's current mood: {mood}"
    message_body = f"Hi! Your pet seems to be feeling {mood} right now."
    send_email_notification(recipient_email, subject, message_body)


# Step 1: Extract Dataset from ZIP
def extract_zip(zip_file_path, extract_to):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)


# Step 2: Load and Preprocess Audio Samples
def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)  # Resample to 16 kHz
    return signal


def preprocess_audio(signal):
    if len(signal) < 16000:
        signal = np.pad(signal, (0, 16000 - len(signal)), 'constant')
    return signal[:16000]  # Trim or pad to 1 second


def load_dataset(dataset_path):
    data = []
    labels = []

    for file in os.listdir(dataset_path):
        if file.endswith('.wav'):
            file_path = os.path.join(dataset_path, file)
            audio_signal = preprocess_audio(load_audio(file_path))
            data.append(audio_signal)

            # Map the file name to a higher-level mood (customize this mapping)
            if "growling" in file:
                labels.append("aggressive")  # Example mapping
            elif "whining" in file:
                labels.append("hungry")  # Example mapping
            elif "barking" in file:
                labels.append("happy")  # Example mapping
            else:
                # Add a default label for files that don't match the above conditions
                labels.append("unknown")  # You can adjust the default label as needed

    return np.array(data), np.array(labels)


# Step 3: Encode Labels

!pip install tensorflow tensorflow-hub keras scikit-learn joblib sounddevice librosa

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import os
import zipfile
import numpy as np
import librosa
import glob
import tensorflow as tf
import tensorflow_hub as hub
from keras import layers, models
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import joblib
import sounddevice as sd
from tensorflow.keras.layers import Input, Lambda  # Import necessary modules
from tensorflow.keras.models import Model
from tensorflow.keras.utils import register_keras_serializable  # Import register_keras_serializable


def send_email_notification(recipient_email, subject, message_body):
    # Email credentials (Use your own credentials)
    sender_email = "1032221997@mitwpu.edu.in"
    sender_password = "akiu wdzm yyea sarn"  # Or App Password if 2FA is enabled

    # Setup the email details
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the message body
    msg.attach(MIMEText(message_body, 'plain'))

    try:
        # Establish a secure connection with Gmail's SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Enable encryption for security
        server.login(sender_email, sender_password)  # Login to the email server

        # Send the email
        server.sendmail(sender_email, recipient_email, msg.as_string())
        print(f"Email sent to {recipient_email}")

    except Exception as e:
        print(f"Failed to send email: {e}")
    finally:
        server.quit()  # Close the SMTP connection


# Example usage
def notify_pet_mood(mood):
    recipient_email = "tanishauchakraborty@gmail.com"  # Replace with user's email
    subject = f"Your pet's current mood: {mood}"
    message_body = f"Hi! Your pet seems to be feeling {mood} right now."
    send_email_notification(recipient_email, subject, message_body)


# Step 1: Extract Dataset from ZIP
def extract_zip(zip_file_path, extract_to):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)


# Step 2: Load and Preprocess Audio Samples
def load_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)  # Resample to 16 kHz
    return signal


def preprocess_audio(signal):
    if len(signal) < 16000:
        signal = np.pad(signal, (0, 16000 - len(signal)), 'constant')
    return signal[:16000]  # Trim or pad to 1 second


def load_dataset(dataset_path):
    data = []
    labels = []

    for file in os.listdir(dataset_path):
        if file.endswith('.wav'):
            file_path = os.path.join(dataset_path, file)
            audio_signal = preprocess_audio(load_audio(file_path))
            data.append(audio_signal)

            # Map the file name to a higher-level mood (customize this mapping)
            if "growling" in file:
                labels.append("aggressive")  # Example mapping
            elif "whining" in file:
                labels.append("hungry")  # Example mapping
            elif "barking" in file:
                labels.append("happy")  # Example mapping
            else:
                # Add a default label for files that don't match the above conditions
                labels.append("unknown")  # You can adjust the default label as needed

    return np.array(data), np.array(labels)


# Step 3: Encode Labels
def encode_labels(labels):
    le = LabelEncoder()
    integer_encoded = le.fit_transform(labels)
    return integer_encoded

In [ ]:
from google.colab import files
uploaded=files.upload()


Saving 2.csv to 2.csv


In [ ]:
import pandas as pd
import numpy as np
import wave

# Load the ThingSpeak data from the CSV
data = pd.read_csv('2.csv')  # Replace with the path to your CSV file

# Take the last 5 readings from the sound data
sound_data = data['field2'].fillna(0).astype(int).tail(5)

# Define parameters for WAV
sample_rate = 44100  # Standard sample rate for audio
amplitude = 32767  # Max amplitude for 16-bit audio

# Repeat the readings to ensure the audio file duration if needed
# This line repeats each sample for 8820 frames, creating roughly 0.2s per sample in the WAV file.
audio_data = np.tile(np.int16((sound_data / sound_data.max()) * amplitude), 8820)

# Save to a WAV file
with wave.open('sound_data.wav', 'w') as wav_file:
    wav_file.setnchannels(1)  # Mono sound
    wav_file.setsampwidth(2)  # 2 bytes per sample (16-bit audio)
    wav_file.setframerate(sample_rate)
    wav_file.writeframes(audio_data.tobytes())

print("WAV file created successfully.")


WAV file created successfully.
